In [2]:
import pandas as pd

df = pd.read_csv('data/train_data_categories.csv')
df = df[df['video_id'] != 'b4d70f82038d1d97f1b3ce2a493d12c8']

def get_first_level(x):
    return x.split(':')[0].strip()

def get_second_level(x):
    if x.count(':') > 0:
        return x.split(':')[1].strip()
    else:
        return '-'
    
def get_third_level(x):
    if x.count(':') == 2:
        return x.split(':')[2].strip()
    else:
        return '-'

df['first_level'] = df['tags'].apply(get_first_level)
df['second_level'] = df['tags'].apply(get_second_level)
df['third_level'] = df['tags'].apply(get_third_level)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
import re


def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text


df['text_for_classification'] = df['title']
df['cleaned_title'] = df['text_for_classification'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], df['first_level'], test_size=0.4, random_state=42)

catboost_model = CatBoostClassifier(
    iterations=2000, 
    # learning_rate=0.1,
    depth=6,
    verbose=100,
    random_seed=42,
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1,2),
        max_features=300,
        min_df=4,
    )),
    ('logreg', catboost_model)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 micro: {f1_micro}')
print(f'F1 macro: {f1_macro}')

Learning rate set to 0.043018
0:	learn: 4.1120077	total: 116ms	remaining: 3m 52s
100:	learn: 2.4163664	total: 2.87s	remaining: 54s
200:	learn: 2.0823534	total: 5.52s	remaining: 49.4s
300:	learn: 1.8614763	total: 8.14s	remaining: 46s
400:	learn: 1.7149731	total: 10.8s	remaining: 43.3s
500:	learn: 1.6154172	total: 13.5s	remaining: 40.4s
600:	learn: 1.5310855	total: 16.2s	remaining: 37.6s
700:	learn: 1.4576241	total: 18.9s	remaining: 34.9s
800:	learn: 1.3946347	total: 21.5s	remaining: 32.2s
900:	learn: 1.3350690	total: 24.1s	remaining: 29.5s
1000:	learn: 1.2829202	total: 26.8s	remaining: 26.8s
1100:	learn: 1.2346694	total: 29.6s	remaining: 24.2s
1200:	learn: 1.1908101	total: 32.3s	remaining: 21.5s
1300:	learn: 1.1494325	total: 34.9s	remaining: 18.7s
1400:	learn: 1.1106193	total: 37.5s	remaining: 16s
1500:	learn: 1.0754595	total: 40.1s	remaining: 13.3s
1600:	learn: 1.0430409	total: 42.7s	remaining: 10.6s
1700:	learn: 1.0134519	total: 45.3s	remaining: 7.96s
1800:	learn: 0.9841801	total: 47.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import re

# Функция для очистки текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)
    return text

# Применение очистки текста к колонке 'title'
df['cleaned_title'] = df['title'].apply(clean_text)

# Сплит меток по запятой и удаление лишних пробелов
df['labels'] = df['first_level'].apply(lambda x: [label.strip() for label in x.split(',')])

# Преобразование меток в формат, подходящий для многометочного обучения
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_title'], y, test_size=0.4, random_state=42)

tfidf_params = {
    # 'max_df': 0.9,
    'min_df': 5,
    'ngram_range': (1,2),
    'max_features': 5000
}

catboost_model = CatBoostClassifier(
    iterations=1000, 
    learning_rate=0.1,
    depth=6,
    verbose=False,
    random_seed=42,
)

log_reg_model = LogisticRegression()


# Создание пайплайна
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(**tfidf_params)),
    ('catboost', OneVsRestClassifier(log_reg_model))
])

# Обучение модели
pipeline.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = pipeline.predict(X_test)

# Оценка точности и F1-меры
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

# Доступ к словарю tfidf и количество слов
tfidf = pipeline.named_steps['tfidf']
vocab_size = len(tfidf.vocabulary_)
print(f'TFIDF Vocabulary Size: {vocab_size}')

Accuracy: 0.4643
F1 Score: 0.5989
TFIDF Vocabulary Size: 241


/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 6 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 11 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 12 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(
/home/dmitrii/anaconda3/envs/venv_ilona/lib/python3.11/site-packages/sklearn/multiclass.py:90: UserWarning: Label not 36 is pre

In [125]:
for x,y in df['tags'].value_counts().to_dict().items():
    print(x, y)

Массовая культура: Юмор и сатира 224
Религия и духовность: астрология 57
Еда и напитки: Кулинария 41
Массовая культура, Карьера 37
Массовая культура 24
Путешествия, События и достопримечательности: Исторические места и достопримечательности 22
Массовая культура: Юмор и сатира, Семья и отношения 22
Спорт: Рыбалка 20
Дом и сад: Дизайн интерьера 20
Массовая культура, Музыка и аудио 18
Транспорт, Спорт: Автогонки, События и достопримечательности: Спортивные события 17
Еда и напитки: Кулинария, Массовая культура 16
Массовая культура, Спорт 15
Музыка и аудио: Комедия и стендап (Музыка и аудио), События и достопримечательности: Концерты и музыкальные мероприятия 14
Массовая культура: Отношения знаменитостей 13
Религия и духовность: астрология, События и достопримечательности: Комедия и стендап 12
Массовая культура, Хобби и интересы: Декоративно-прикладное искусство 11
Семья и отношения 11
Фильмы и анимация: Семейные и детские фильмы, Фильмы и анимация: Фильмы и анимация  11
Массовая культура,

In [121]:
df[df['first_level'] == 'Массовая культура, Карьера']

,video_id,title,description,tags,first_level,second_level,third_level,cleaned_title,text_for_classification
91,51304cc9ff8e942ce481282b6952dd86,Артмеханика. Концерт группы TIHOTIHO,Концерт группы TIHOTIHO,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,концерт группы tihotiho,Концерт группы TIHOTIHO
196,02b797508b9ffc941887703dfb17d365,Артмеханика. Сезон 2. Выпуск 6. Эволюция веду...,Влад Маленко и Ольга Ершова разбираются в том...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,влад маленко и ольга ершова разбираются в том...,Влад Маленко и Ольга Ершова разбираются в том...
220,034753227cf7772245e3fa97c5274932,"Артмеханика. Игра ""Лицо рекламы"".","Игра ""Лицо рекламы""","Массовая культура, Карьера","Массовая культура, Карьера",-,-,игра лицо рекламы,"Игра ""Лицо рекламы"""
234,13aba71d6501df8db686919affbc9b72,Артмеханика. Интервью. Денис Рогов.,Что такое метавселенная? Где мы можем ощутить ...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что такое метавселенная где мы можем ощутить к...,Что такое метавселенная? Где мы можем ощутить ...
236,63d9eab25494382081b6ad100efa7ad4,Артмеханика. Концерт группы Dabro.,Все самые лучшие хиты группы Dabro.,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,все самые лучшие хиты группы dabro,Все самые лучшие хиты группы Dabro.
258,d42d194ba6c4e674a6a625106a76fb1e,Артмеханика. Интервью. Степан Липгарт.,Что собой представляет современная архитектура...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,что собой представляет современная архитектура...,Что собой представляет современная архитектура...
259,343470324af6902808606cf1a7503575,Артмеханика. Интервью с сёстрами Набока.,Как сёстры Набока покорили известных актёров и...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,как сёстры набока покорили известных актёров и...,Как сёстры Набока покорили известных актёров и...
261,7413b5707b2119fc392f0ac00cc2a8a1,Артмеханика. Интервью с Анастасией Моргун и Юр...,Зачем сериал Санта-Барбара вернулся к нам спус...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,зачем сериал сантабарбара вернулся к нам спуст...,Зачем сериал Санта-Барбара вернулся к нам спус...
265,9444666b1f0d17c502e46ecbaad08b67,Артмеханика. Сезон 2. Выпуск 15. Подлинное иск...,В нашей студии стало жарко! “Арт Механика” реш...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,в нашей студии стало жарко арт механика решила...,В нашей студии стало жарко! “Арт Механика” реш...
267,c477b6170b5991c1550032c394d93fd0,Артмеханика. Интервью с Евгением Чесом.,Куда приведет тенденция к тотальной кастомизац...,"Массовая культура, Карьера","Массовая культура, Карьера",-,-,куда приведет тенденция к тотальной кастомизац...,Куда приведет тенденция к тотальной кастомизац...
